In [1]:
import xarray as xr
import numcodecs

In [ ]:
# Notebook to produce Zarr files from the Spatial metrics to be displayed with the CarbonPlan Minimaps component
# no pyramids -> one zoom level only

In [63]:
ds = xr.open_dataset("Global_marsai_MAE_MAP_leadtimes.nc")

<xarray.Dataset> Size: 2GB
Dimensions:    (latitude: 721, longitude: 1440, time: 41)
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * time       (time) float64 328B 0.0 1.0 2.0 3.0 4.0 ... 37.0 38.0 39.0 40.0
    number     int64 8B ...
    expver     (time) <U4 656B ...
Data variables:
    t2m        (time, latitude, longitude) float64 341MB ...
    q          (time, latitude, longitude) float64 341MB ...
    u10        (time, latitude, longitude) float64 341MB ...
    v10        (time, latitude, longitude) float64 341MB ...
    msl        (time, latitude, longitude) float64 341MB ...

In [64]:
# Drop 'number' and 'expver' coordinates if they exist
for coord in ['number', 'expver', 'isobaricInhPa', 'surface', 'meanSea', 'step']:
   if coord in ds.coords:
      ds = ds.drop_vars(coord)

In [65]:
ds = ds.rename({"latitude": "y", "longitude": "x"})

In [67]:
# Convert float64 variables to float32 
ds = ds.assign({var: ds[var].astype('float32') for var in ds.data_vars if ds[var].dtype == 'float64'})

In [68]:
for coord in ['x', 'y', 'time']:
    if coord in ds.coords and ds.coords[coord].dtype == 'float64':
        ds = ds.assign_coords({coord: ds.coords[coord].astype('float32')})

In [70]:
# Flip along the 'y' dimension
# minimaps needs lat values between -90 to 90 and lon -180 to 180
ds = ds.sel(y=ds.y[::-1])

<xarray.Dataset> Size: 851MB
Dimensions:  (y: 721, x: 1440, time: 41)
Coordinates:
  * y        (y) float32 3kB -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * x        (x) float32 6kB -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * time     (time) float32 164B 0.0 1.0 2.0 3.0 4.0 ... 37.0 38.0 39.0 40.0
Data variables:
    t2m      (time, y, x) float32 170MB 1.387 1.387 1.387 ... 2.366 2.366 2.366
    q        (time, y, x) float32 170MB 0.01713 0.01713 ... 0.3893 0.3893
    u10      (time, y, x) float32 170MB 0.4191 0.4191 0.4191 ... 3.571 3.571
    v10      (time, y, x) float32 170MB 0.8157 0.8157 0.8157 ... 2.526 2.526
    msl      (time, y, x) float32 170MB 1.143 1.143 1.143 ... 6.524 6.524 6.524

In [71]:
ds = ds.transpose('time', 'y', 'x')

<xarray.Dataset> Size: 851MB
Dimensions:  (y: 721, x: 1440, time: 41)
Coordinates:
  * y        (y) float32 3kB -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * x        (x) float32 6kB -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * time     (time) float32 164B 0.0 1.0 2.0 3.0 4.0 ... 37.0 38.0 39.0 40.0
Data variables:
    t2m      (time, y, x) float32 170MB 1.387 1.387 1.387 ... 2.366 2.366 2.366
    q        (time, y, x) float32 170MB 0.01713 0.01713 ... 0.3893 0.3893
    u10      (time, y, x) float32 170MB 0.4191 0.4191 0.4191 ... 3.571 3.571
    v10      (time, y, x) float32 170MB 0.8157 0.8157 0.8157 ... 2.526 2.526
    msl      (time, y, x) float32 170MB 1.143 1.143 1.143 ... 6.524 6.524 6.524

In [ ]:
# produce zarr with zlib compression which is supported by the zarr-js library (used in the next.js app)
# blosc compression is not supported by the zarr-js library
# latitudes must be arranged from -90 to 90 (longitudes from -180 to 180)

In [74]:
ds_chunked = ds.chunk({'time': 1, 'y': 721, 'x': 1440})

# Define zlib compression for all data variables
encoding = {
    var: {'compressor': numcodecs.Zlib(level=1)}
    for var in ds_chunked.data_vars
}

# Export to Zarr with zlib compression
ds_chunked.to_zarr('MinimapsZarrFiles/Global_marsai_MAE_MAP_leadtimes.zarr', mode='w', encoding=encoding)